# Time series forecasting algorithms in PyAF

This document describes the algorithmic aspects of time series forecasting in PyAF. 
We will describe:
1. The overall algorithm
2. The detail of the signal decomposition
3. The machine learning aspects
4. Model Customization. Advanced usage/control of the algorithms.
5. Hierarchical forecasting.

Warning : This document ins intended for advanced uses of PyAF. The aspects described here are not useful in a typical forecasting use case.


## Generic Algorithmic Choices

PyAF uses a machine learning approach to forecasting. A lot of time series models are generated and their forecasting quality is compared on a validation dataset (most recent part of the whole signal). 
To summarize, PyAF is performing a competition between a large set of possible models/hypothesis and selecting the best to perform the final forecast.

the models used/tested in PyAF are signal decompositions generated on the fly internally. An additive signal decomposition is the sum of a **trend** (long term) , **periodic** and an **irregular** component as described in http://en.wikipedia.org/wiki/Decomposition_of_time_series

Fro the theoretical aspects, PyAF uses standard methods and a lot of best practices. When not specified, the theoretical apsects are mainly inspired by the excellent approach used in [Rob J Hyndman and George Athanasopoulos book](http://www.otexts.org/fpp3/).

PyAF generates tens of possible decomposition for the input signal and outputs the best. One can control the amount/types of decompositions, enable/disable such of such component,  and review the performance of each decomposition internally.

In addition to the decomposition , PyAF allows a whole set of possible **signal transformations** performed in a pre-processing phase (before decomposition) and a  post-processing step (after forecasting).

PyAF performs the forecasting task of a signal $X_t$ in three steps described below:

1. Signal Transformation : $$ Y_t = \phi(X_t) $$
2. Decomposition of the transformed signal (non-additive decompositions are also supported): $$ \hat{Y_t} = T_t + C_t + I_t $$
3. Back transformation of the forecast : $$ \hat{X_t}  = \phi^{-1}(\hat{Y_t}) $$


## Signal Decompositions with PyAF

 PyAF supports the following operations :

In [8]:
lKnownDecompositions = ['T+S+R' , 'TS+R', 'TSR']

lKnownTransformations = ['None', 'Difference', 'RelativeDifference','Integration', 'BoxCox', 'Quantization', 'Logit', 'Fisher', 'Anscombe'];

lKnownTrends = ['ConstantTrend', 'Lag1Trend', 'LinearTrend', 'PolyTrend','MovingAverage', 'MovingMedian'];

lKnownPeriodics = ['NoCycle', 'BestCycle',
                   'Seasonal_MonthOfYear' ,
                   'Seasonal_Second' ,
                   'Seasonal_Minute' ,
                   'Seasonal_Hour' ,
                   'Seasonal_HourOfWeek' ,
                   'Seasonal_TwoHourOfWeek' ,
                   'Seasonal_ThreeHourOfWeek' ,
                   'Seasonal_FourHourOfWeek' ,
                   'Seasonal_SixHourOfWeek' ,
                   'Seasonal_EightHourOfWeek' ,
                   'Seasonal_TwelveHourOfWeek' ,
                   'Seasonal_DayOfWeek' ,
                   'Seasonal_DayOfMonth',
                   'Seasonal_DayOfYear',
                   'Seasonal_WeekOfMonth',
                   'Seasonal_DayOfNthWeekOfMonth',
                   'Seasonal_WeekOfYear']

# 'X' at the end , means with exogenous data (when available).
lKnownAutoRegressions = ['NoAR' , 
                         'AR' , 'ARX' , 
                         'SVR', 'SVRX', 
                         'MLP' , 'MLPX',
                         'LSTM' , 'LSTMX',
                         'XGB' , 'XGBX' , 
                         'CROSTON', 
                         'LGB', 'LGBX'];





In [9]:
import numpy as np
import pandas as pd

N = len(lKnownPeriodics)
df = pd.DataFrame(index = range(N))
df['Transformation'] = lKnownTransformations + [""]*(N - len(lKnownTransformations))
df['Decomposition'] = lKnownDecompositions + [""]*(N - len(lKnownDecompositions))
df['Trend'] = lKnownTrends + [""]*(N - len(lKnownTrends))
df['Periodics'] = lKnownPeriodics + [""]*(N - len(lKnownPeriodics))
df['Irregular'] = lKnownAutoRegressions + [""]*(N - len(lKnownAutoRegressions))

df.head(N)

,Transformation,Decomposition,Trend,Periodics,Irregular
0,None,T+S+R,ConstantTrend,NoCycle,NoAR
1,Difference,TS+R,Lag1Trend,BestCycle,AR
2,RelativeDifference,TSR,LinearTrend,Seasonal_MonthOfYear,ARX
3,Integration,,PolyTrend,Seasonal_Second,SVR
4,BoxCox,,MovingAverage,Seasonal_Minute,SVRX
5,Quantization,,MovingMedian,Seasonal_Hour,MLP
6,Logit,,,Seasonal_HourOfWeek,MLPX
7,Fisher,,,Seasonal_TwoHourOfWeek,LSTM
8,Anscombe,,,Seasonal_ThreeHourOfWeek,LSTMX
9,,,,Seasonal_FourHourOfWeek,XGB


PyAF tests every possible combination of these components, one of each column in the previous table. The user has the possibility to restrict each column and perform the training with the selected items (see customization below). 

Some algorithmic choices are made in pyaf (pandas operations are mainly column-based, dataframes are shared between various data processing steps, every signal transformation goes on specific thread). these choices make it possible to handle thousands of possible models. 

All the available CPUs can be used to accelerate PyAF training (minus a few to maintain a usable system ;). PyaF was tested on a machine with a xeon-phi CPU (https://github.com/antoinecarme/xeon-phi-data) which can process 256 threads simultaneously. 

By default, PyaF uses only the most common components (the first 3 or 4 of each column). The remaining components are optional and can be activated before training (see customization below)

We now give the possible values for each column , with their formula and specify when optional : 

### Transformations

1. None : $Y_t = \phi(X_t) = X_t $
2. Difference : $Y_t =  \phi(X_t) = X_t  - X_{t-1} $
3. Relative Difference : $Y_t =  \phi(X_t) = \frac{X_t  - X_{t-1}}{X_{t-1}} $
4. Integration : $Y_t =  \phi(X_t) = \sum_{s=0}^{s=t} X_s $

Optional Transformations:

5. BoxCox : $Y_t =  \phi(X_t) = \frac{X_t^\lambda - 1}{\lambda} $ (https://en.wikipedia.org/wiki/Power_transform#Box%E2%80%93Cox_transformation)
6. Quantization : $Y_t =  \phi(X_t) = quantile(X_t) $
7. Logit : $Y_t =  \phi(X_t) = \frac{1}{1 + e^{-X_t}} $ (https://en.wikipedia.org/wiki/Logit)
8. Anscombe : $Y_t =  \phi(X_t) = 2 \sqrt{X_t + \frac{3}{8}} $ (https://en.wikipedia.org/wiki/Anscombe_transform)
9. Fisher : $Y_t =  \phi(X_t) = atanh(X_t) $ (https://en.wikipedia.org/wiki/Fisher_transformation)

### Trends



1. ConstantTrend : $T_t = a $
2. LinearTrend : $T_t = a t + b$
3. PolyTrend : $T_t = a t^2 + b t + c$
4. Lag1Trend : $T_t = Y_{t-1} $

Optional Trends

5. MovingAverage : $T_t = \sum_{s=t-k}^{s=t-1} Y_s$
6. MovingMedian : $T_t = median(Y_{t-k}, \dots, Y_{t-1})$



### Periodicities


1. None: $C_t = 0 $
2. BestCycle (extracted automatically, $p$ is estimated from data) : $ C_t = C_{t-p}$
3. Seasonality (depends on date parts) : $ C_t = minute(t) , hour(t), etc$



### Irregular Components

These models are built of the residues of trend and cycles:

$Z_t = Y_t - T_t - C_t $

We consider here some models based on the residue lags ($Lag(Z, k)_t = Z_{t-k}$).

The models described here as implemented using external libraries, either scikit-learn (AR, ARX, SVR), pytorch or keras (MLP, LSTM), XGBoost, LightGBM, ...

The default models are :

1. None : $I_t = 0$
2. Autoregressive (AR) model :  $I_t = a_0 + \sum_{k=1}^{k=p} a_k Z_{t-k}$
3. Autoregressive with Exogenous data (ARX) model :  $I_t = a_0 + \sum_{k=1}^{k=p} a_k Z_{t-k} + \sum_{k=1}^{k=p} b_k Exog_{t-k} $

Experimental Models : 

4. Support Vector Regression  (SVR) : $I_t = RBFKernel(target = Z_t , inputs = \{Z_{t-1} , \dots, Z_{t-p}\})$
5. MultiLayer Perceptron  (MLP) : $I_t = MLPRegressor(target = Z_t , inputs = \{Z_{t-1} , \dots, Z_{t-p}\})$
6. LSTM : $I_t = LSTMRegressor(target = Z_t , inputs = \{Z_{t-1} , \dots, Z_{t-p}\})$
7. XGB : $I_t = XGBoostRegressor(target = Z_t , inputs = \{Z_{t-1} , \dots, Z_{t-p}\})$
8. LGB : $I_t = LightGBMRegressor(target = Z_t , inputs = \{Z_{t-1} , \dots, Z_{t-p}\})$
6. CROSTON : $I_t = Croston(target = Z_t , inputs = \{Z_{t-1} , \dots, Z_{t-p}\})$

The parameter $p$ repesents the dependency on the past. It can be customized.

All these algorithms also support the usage of exogenous variables past (inputs = $\{Z_{t-1} , \dots, Z_{t-p}, Exog_{t-1}, \dots, Exog_{t-p}  \}$ instead of inputs = $\{Z_{t-1} , \dots, Z_{t-p}\}$), The naming convention is to use a 'X' suffix (i.e : 'LSTMX' is 'LSTM' with exogenous inputs)

## Model Customization

PyAF, by default, supports two modes of operatrion : **Fast Mode** and **Slow Mode**.

In Fast Mode, actviated by default, an effort was made to give the most of PyAF in an acceptable coputation time. 
Only the non-optional models are activated in this mode.

In Slow Mode, all the models are activated , PyAF achieves better models but does no guarantee on computation times.

Some benchmarking and a lot of tests are performed to ensure that PyAF, even in slow mode, generates acceptable models within acceptable training times.

The user can customize PyAF to build intermediate models and restrict PyAF to use some categories of models. PyAF options allow this customization (https://github.com/antoinecarme/pyaf/blob/b614bfe24043a47e1aaf7cff36a1447b5dc431f6/pyaf/TS/Options.py#L13).

We give here an example of a model that only uses linear trends, some seasonal components and an xgboost-based irregular component

In [13]:
import pandas as pd
import numpy as np
import pyaf.ForecastEngine as autof
import pyaf.Bench.TS_datasets as tsds

get_ipython().magic('matplotlib inline')

b1 = tsds.load_ozone()


lEngine = autof.cForecastEngine()
lEngine

H = b1.mHorizon;
lEngine.mOptions.set_active_transformations(['None']);
lEngine.mOptions.set_active_trends(['LinearTrend']);
lEngine.mOptions.set_active_periodics(['Seasonal_MonthOfYear' , 'Seasonal_Hour' , 'Seasonal_HourOfWeek']);
lEngine.mOptions.set_active_autoregressions(['XGB']);

# lEngine.mOptions.mDebugPerformance = True;
lEngine.train(b1.mPastData , b1.mTimeVar , b1.mSignalVar, H);
lEngine.getModelInfo();
lEngine.mSignalDecomposition.mTrPerfDetails.head()


INFO:pyaf.timing:('OPERATION_START', ('SIGNAL_TRAINING', {'Signals': ['Ozone'], 'Transformations': [('Ozone', 'None', '_', 'T+S+R')], 'Cores': 1}))
INFO:pyaf.timing:('OPERATION_START', ('TRAINING', {'Signal': 'Ozone', 'Transformation': '_Ozone'}))
INFO:pyaf.timing:('OPERATION_END_ELAPSED', 0.115, ('TRAINING', {'Signal': 'Ozone', 'Transformation': '_Ozone'}))
INFO:pyaf.timing:('OPERATION_END_ELAPSED', 0.117, ('SIGNAL_TRAINING', {'Signals': ['Ozone'], 'Transformations': [('Ozone', 'None', '_', 'T+S+R')], 'Cores': 1}))
INFO:pyaf.timing:('OPERATION_START', ('FINALIZE_TRAINING', {'Signals': ['Ozone'], 'Transformations': [('Ozone', [('Ozone', 'None', '_', 'T+S+R')])], 'Cores': 1}))
INFO:pyaf.timing:('OPERATION_START', ('MODEL_SELECTION', {'Signal': 'Ozone', 'Transformations': [('Ozone', 'None', '_', 'T+S+R')]}))
INFO:pyaf.timing:('OPERATION_END_ELAPSED', 0.006, ('MODEL_SELECTION', {'Signal': 'Ozone', 'Transformations': [('Ozone', 'None', '_', 'T+S+R')]}))
INFO:pyaf.timing:('OPERATION_START',

     Month  Ozone       Time
0  1955-01    2.7 1955-01-01
1  1955-02    2.0 1955-02-01
2  1955-03    3.6 1955-03-01
3  1955-04    5.0 1955-04-01
4  1955-05    6.5 1955-05-01


INFO:pyaf.timing:('OPERATION_END_ELAPSED', 0.381, ('COMPUTE_PREDICTION_INTERVALS', {'Signal': 'Ozone'}))
INFO:pyaf.timing:('OPERATION_END_ELAPSED', 0.423, ('FINALIZE_TRAINING', {'Signals': ['Ozone'], 'Transformations': [('Ozone', [('Ozone', 'None', '_', 'T+S+R')])], 'Cores': 1}))
INFO:pyaf.std:TIME_DETAIL TimeVariable='Time' TimeMin=1955-01-01T00:00:00.000000 TimeMax=1967-09-01T00:00:00.000000 TimeDelta=<DateOffset: months=1> Horizon=12
INFO:pyaf.std:SIGNAL_DETAIL_ORIG SignalVariable='Ozone' Length=204  Min=1.2 Max=8.7  Mean=3.835784 StdDev=1.491559
INFO:pyaf.std:SIGNAL_DETAIL_TRANSFORMED TransformedSignalVariable='_Ozone' Min=0.0 Max=1.0  Mean=0.351438 StdDev=0.198875
INFO:pyaf.std:DECOMPOSITION_TYPE 'T+S+R'
INFO:pyaf.std:BEST_TRANSOFORMATION_TYPE '_'
INFO:pyaf.std:BEST_DECOMPOSITION  '_Ozone_LinearTrend_residue_Seasonal_MonthOfYear_residue_XGB(51)' [LinearTrend + Seasonal_MonthOfYear + XGB]
INFO:pyaf.std:TREND_DETAIL '_Ozone_LinearTrend' [LinearTrend]
INFO:pyaf.std:CYCLE_DETAIL '_Ozo

,Split,Transformation,DecompositionType,Model,DetailedFormula,Category,Complexity,FitMAPE,ForecastMAPE,TestMAPE
0,None,_Ozone,T+S+R,_Ozone_LinearTrend_residue_Seasonal_MonthOfYea...,"(_Ozone, T+S+R, None, _Ozone_LinearTrend_resid...",NoTransf_LinearTrend_Seasonal_MonthOfYear_XGB,51.0,0.1173,0.1846,0.224
